# Mentoría 'de cómo clasificar en géneros a las canciones'
## Práctico III : Introducción al aprendizaje automático

**Antes de empezar:**
- [Instalar spaCy y el modelo de lenguaje con el que van a trabajar](https://spacy.io/models#quickstart)

**Consideraciones:**
- Se evalúa el estilo y prolijidad del código.
- Se permite hacer trabajo extra, siempre y cuando las actividades básicas estén resueltas.

**Recomendación:**
- Hay muchos ejemplos de código en internet, no se sientan obligados a implementar todo desde cero.

### Librerías

In [ ]:
!pip3 install spotipy
!pip3 install pandas
!pip3 install spacy
!pip3 install pymusixmatch
!pip3 install nltk
!pip3 install sklearn

# Agregar las librerías extra que se utilicen en esta celda y la siguiente

### Dependencias y acceso a APIs

In [5]:
import pandas as pd
import numpy as np
import spotipy
import spacy
from spotipy.oauth2 import SpotifyClientCredentials
from musixmatch import Musixmatch
import seaborn as sns
import tqdm
import plotly.express as px
import plotly.graph_objects as go

client_id = '46b333d567314a89a6254b6c6b054be6'
client_secret = '9d922c3613e441518349dcf55f7d5853'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# es = es_core_web_sm
nlp = spacy.load("es") # completar con el modelo que van a utilizar

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
musixmatch = Musixmatch('1aa5272f4402bf2f082ad2f3958c2c62') # se puede reemplazar por otra API si da mejores resultados

In [4]:
sns.set_context(context='paper')

### 1) Recopilar los datos obtenidos en los prácticos anteriores

Para esta parte consideraremos [la playlist colaborativa de la mentoría](https://open.spotify.com/playlist/2IuD0qZb14cji5y52crdsO?si=nfHRPDquQRyotEcXc4tG7Q), de esta obtendremos:
- Las features del audio de las canciones
- Las features textuales de sus letras

Además es necesario aplicar el mismo preprocesamiento que aplicamos en los prácticos anteriores para ambos tipos de features (el preprocesamiento del p1 a las features de audio y el de p2 al de features textuales) y obtener el género de cada canción, que en caso de ser más de uno para una canción el equipo deberá discutir una estrategia para estos casos y comentarla en el informe.

Luego, se separará al dataset resultante en **X** e **y**, donde:
- X es el conjunto de features
- y es la etiqueta, en este caso el género de la canción, que deberá ser codificado en valores del tipo **int**

Por último, se dividirá a estos dos conjuntos en los splits **train** y **test**

**Recomendaciones:**
- Obtener las features por separado y hacer un join de los datasets.
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)
- Si usan features categóricas, ENCODEARLAS!

# Dataset

## Music Features

In [6]:
#Aux funcs
def genres_by_artist_id(id): #id: str
    artist = sp.artist(id)
    genres = artist['genres']
    return genres    #genres: List[str]

def songs_from_album_id(album_id):
    songs = []
    album = sp.album(album_id)
    artist = album['artists'][0]['name']
    for item in album['tracks']['items']:
        track = {}
        track["song_name"] = item['name']
        track["song_id"] = item['id']
        track["album_name"] = album['name']
        track["album_id"] = album["id"]
        audio_features = sp.audio_features(track["song_id"])
        track["audio_features"] = audio_features[0]
        track["artist"] = artist
        songs.append(track)
    return songs    #songs:List[dict]

def add_track(track_id, songs): #track_id:str, songs:List[dict]
    track = sp.track(track_id)
    audio_features = sp.audio_features(track_id)
    row = {}
    row["song_name"] = track['name']
    row["song_id"] = track['id']
    row["album_name"] = track['album']['name']
    row["album_id"] = track['album']['id']
    row["audio_features"] = audio_features[0]
    songs.append(row)
    return songs


In [109]:
PLAYLIST_ID = "2IuD0qZb14cji5y52crdsO"
songs = []
def add_songs_of_playlist(playlist_id,songs_array):
    print("This may take a while")
    offset = 0
    playlist = sp.playlist_tracks(playlist_id)
    for i in tqdm.tqdm(range(playlist["total"])):
        j = i%100
        if i == 99:
            offset += 1
            playlist = sp.playlist_tracks("2IuD0qZb14cji5y52crdsO", offset=i+offset,limit=100)
        add_track(playlist["items"][j]["track"]["id"],songs_array)
    return


add_songs_of_playlist(PLAYLIST_ID,songs)


This may take a while


100%|██████████| 1188/1188 [07:58<00:00,  2.48it/s]


In [118]:
songs_original_df = pd.DataFrame(songs)
songs_original_df.sample(5)

,album_id,album_name,audio_features,song_id,song_name
3,1QXxmsxolhkqiFtI1mpX4i,Sus 16 Grandes Exitos,"{'danceability': 0.499, 'energy': 0.648, 'key'...",2kfSFdq2h0xLXq01em1zc7,La Gata Bajo la Lluvia
851,22L8oOyAV6QQXYpXrE00nX,Gracias a Dios,"{'danceability': 0.479, 'energy': 0.664, 'key'...",4huDyihm2NyXchfvIK1aFg,Calla Chiquitín
311,50PvQ5VYnLUuXd1DZIJBhN,Cuarteto es La Mona,"{'danceability': 0.749, 'energy': 0.901, 'key'...",5G6Ox8WOURx4ItsGfKLgEn,El sangrión
49,1fLKCHRphOtHLQyK0D4DAy,Milongas,"{'danceability': 0.765, 'energy': 0.297, 'key'...",2hHO1v7obfbSLLPC0YC4vv,Si De Rabia Nomas
1099,0GLHrUclRfg9QQxDMqeQTj,Un Loco Bohemio de la Noche,"{'danceability': 0.51, 'energy': 0.955, 'key':...",2nhEQw6fosFKNbwavSMTl5,Simplemente baila


In [120]:
audio_features_base = pd.DataFrame(list(songs_original_df["audio_features"].values))
audio_features_base.sample(5)

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
335,0.1090,https://api.spotify.com/v1/audio-analysis/7Koy...,0.649,268748,0.410,7KoyEA2CC72dhWnGTU27Bq,0.000000,11,0.223,-17.116,1,0.0335,138.053,4,https://api.spotify.com/v1/tracks/7KoyEA2CC72d...,audio_features,spotify:track:7KoyEA2CC72dhWnGTU27Bq,0.673
302,0.3500,https://api.spotify.com/v1/audio-analysis/6SbD...,0.507,302187,0.877,6SbDsZ7cv06KYxSHkBWmkS,0.000000,2,0.950,-5.963,0,0.2240,150.009,4,https://api.spotify.com/v1/tracks/6SbDsZ7cv06K...,audio_features,spotify:track:6SbDsZ7cv06KYxSHkBWmkS,0.596
1082,0.5500,https://api.spotify.com/v1/audio-analysis/3QXe...,0.696,236173,0.585,3QXevipMyXOI5BeLqtlX1J,0.000000,7,0.110,-6.666,0,0.0571,106.932,4,https://api.spotify.com/v1/tracks/3QXevipMyXOI...,audio_features,spotify:track:3QXevipMyXOI5BeLqtlX1J,0.778
964,0.0599,https://api.spotify.com/v1/audio-analysis/1j6T...,0.305,353080,0.947,1j6TYEdFkJ4B9iOnY3vtuq,0.000002,4,0.886,-2.159,0,0.1010,78.016,4,https://api.spotify.com/v1/tracks/1j6TYEdFkJ4B...,audio_features,spotify:track:1j6TYEdFkJ4B9iOnY3vtuq,0.447
60,0.0333,https://api.spotify.com/v1/audio-analysis/24bl...,0.373,336133,0.588,24bl7udyFxIGgMVxTKzKJP,0.026400,4,0.103,-10.385,0,0.0759,149.084,4,https://api.spotify.com/v1/tracks/24bl7udyFxIG...,audio_features,spotify:track:24bl7udyFxIGgMVxTKzKJP,0.365


## Lyrics

Decisión de diseño
El género es una lista de géneros.

Hay canciones que no tienen un género asociado, en ese caso la api devuelve un []. Cómo tratar ese caso?

Yo propongo usar el género del artista. 

Qué pasa si shakira y metallica colaboran? qué género le ponés? Una decision posible: los géneros de los dos.

distribución de los géneros en el dataset.

### 2) Elegir tres modelos de clasificadores multiclase

Aquí escogeremos tres modelos diferentes y luego compararemos su rendimiento para esta tarea. El procedimiento será el siguiente:
- Inicializar los modelos
- Entrenarlos usando el split **train** de los datos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 3) Informe: Comparar el rendimiento de los modelos

Una vez entrenados los tres modelos, compararemos su rendimiento:
- Correr los modelos usando el split **test**
- Obtener el reporte de clasificación y la matriz de confusión para cada modelo
- Graficar llevando a 2 dimensiones nuestro split **test** pintando con colores diferentes según la etiqueta correspondiente.
- Graficar de manera similar los resultados obtenidos con cada clasificador y sobre esto la función de clasificación obtenida.
- Guardar los modelos usando **pickle**
- Discutir los resultados obtenidos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 4) Tareas adicionales:

Estas tareas servirán para extrapolar un poco el trabajo básico, y también sumarán puntos extra. Deben elegir una o más de las siguientes:
-  Análisis sobre el balance de clases del dataset, balanceo usando **subsampling** u **oversampling** y comparación de resultados vs el modelo básico
- Optimización de hiperparámetros y comparación de resultados vs el modelo básico
- Graficar importancia de features
- Graficar correlación de features

**Recomendación:**
- Hacer varias ahora puede ahorrarles tiempo en el futuro